In [ ]:
import numpy as np
import pandas as pd
import h5py
import pysam
import os
from modisco.visualization import viz_sequence
from modisco import util
from matplotlib import pyplot as plt
import pybedtools
from bs4 import BeautifulSoup

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [ ]:
dttype = 'ATAC'
mode = "counts"
celltype="H1ESC"
modisco_path = '/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/ATAC/'+celltype+'/merge_folds_new/'+mode+'/modisco_old_format.h5'
ppm_dir = ''

htmld="/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/"+dttype+'/'+celltype+"/merge_folds_new/"+mode+"/motifs.html"
tomtom = pd.read_html(htmld)


background=[0.25, 0.25, 0.25, 0.25]

In [ ]:
def trim_motif_new(cwm, motif, trim_threshold=0.20):
    """
    Given the PFM and motif (both L x 4 arrays) (the motif could be the
    PFM itself), trims `motif` by cutting off flanks of low information
    content in `pfm`. `min_ic` is the minimum required information
    content. If specified this trimmed motif will be extended on either
    side by `pad` bases.
    If no base passes the `min_ic` threshold, then no trimming is done.
    """
    
    score = np.sum(np.abs(cwm), axis=1)
    trim_thresh = np.max(score) * trim_threshold  # Cut off anything less than 30% of max score
    pass_inds = np.where(score >= trim_thresh)[0]
    trimmed = motif[np.min(pass_inds): np.max(pass_inds) + 1]
 
    if not trimmed.size:
        return motif
    
    return trimmed

def import_tfmodisco_motifs(tfm_results_path, trim=True, only_pos=True):
    """
    Imports the PFMs to into a dictionary, mapping `(x, y)` to the PFM,
    where `x` is the metacluster index and `y` is the pattern index.
    Arguments:
        `tfm_results_path`: path to HDF5 containing TF-MoDISco results
        `out_dir`: where to save motifs
        `trim`: if True, trim the motif flanks based on information content
        `only_pos`: if True, only return motifs with positive contributions
    Returns the dictionary of PFMs.
    """ 
    pfms = {}
    with h5py.File(tfm_results_path, "r") as f:
        metaclusters = f["metacluster_idx_to_submetacluster_results"]
        num_metaclusters = len(metaclusters.keys())
        for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
            metacluster = metaclusters[metacluster_key]
            if "patterns" not in metacluster["seqlets_to_patterns_result"]:
                continue
            patterns = metacluster["seqlets_to_patterns_result"]["patterns"]
            num_patterns = len(patterns["all_pattern_names"][:])
            for pattern_i, pattern_name in enumerate(patterns["all_pattern_names"][:]):
#                pattern_name = pattern_name.decode()
                pattern_name = pattern_name

                pattern = patterns[pattern_name]
                pfm = pattern["sequence"]["fwd"][:]
                cwm = pattern["task0_contrib_scores"]["fwd"][:]
                
                # Check that the contribution scores are overall positive
                if only_pos and np.sum(cwm) < 0:
                    continue
                    
                if trim:
                    pfm = trim_motif_new(cwm, cwm)
                else:
                    pfm = cwm
                    
                pfms["%d_%d" % (metacluster_i,pattern_i)] = pfm
    return pfms

In [ ]:
pfms = import_tfmodisco_motifs(modisco_path, trim=False)

In [ ]:
for key in pfms:
    f = open(os.path.join(ppm_dir,mode+"_"+key+".pfm"),"w")
    #print(pfms[key])
    np.savetxt(f, pfms[key], fmt='%f')
    f.close()
    

In [ ]:
tomtom[0]

In [ ]:

tomtom[0]["pattern"] = tomtom[0]["pattern"].str.replace("pos_patterns.pattern","0").str.replace("neg_patterns.pattern","1")


In [ ]:
tomtom[0]

In [ ]:
tomtom[0][["pattern","num_seqlets"]].to_csv(os.path.join(ppm_dir,mode+"_counts.csv"),sep=",",index=False, header=False)